# 🔮 cellspell — Cypher Spell (Colab)

Run Cypher queries against Neo4j directly in notebook cells.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sreent/jupyter-query-magics/blob/main/examples/colab_cypher.ipynb)

## Setup: Install and Start Neo4j

In [ ]:
# Install Neo4j server and Java
!apt-get update -qq && apt-get install -y -qq openjdk-17-jdk > /dev/null 2>&1
!wget -q https://dist.neo4j.org/neo4j-community-5.26.0-unix.tar.gz
!tar -xf neo4j-community-5.26.0-unix.tar.gz
!neo4j-community-5.26.0/bin/neo4j-admin dbms set-initial-password test1234
!nohup neo4j-community-5.26.0/bin/neo4j console > /dev/null 2>&1 &

In [ ]:
# Install cellspell
!pip install "cellspell[cypher]" -q

In [ ]:
# Wait for Neo4j to be ready
import time
print("Waiting for Neo4j to start...", end="")
for _ in range(30):
    try:
        import urllib.request
        urllib.request.urlopen("http://localhost:7474", timeout=2)
        print(" Ready!")
        break
    except Exception:
        print(".", end="", flush=True)
        time.sleep(2)
else:
    print(" Timeout — Neo4j may not be running.")

In [ ]:
%load_ext cellspell.cypher

## Connect to Neo4j

In [ ]:
%cypher bolt://localhost:7687 -u neo4j -p test1234

In [ ]:
# Show connection info
%cypher

## Create Sample Data

In [ ]:
%%cypher
CREATE (a:Person {name: 'Alice', age: 30, city: 'Bangkok'}),
       (b:Person {name: 'Bob', age: 25, city: 'London'}),
       (c:Person {name: 'Charlie', age: 35, city: 'Bangkok'}),
       (d:Person {name: 'Diana', age: 28, city: 'Tokyo'}),
       (e:Person {name: 'Eve', age: 32, city: 'London'}),
       (a)-[:KNOWS]->(b),
       (a)-[:KNOWS]->(c),
       (b)-[:KNOWS]->(d),
       (c)-[:KNOWS]->(d),
       (d)-[:KNOWS]->(e)

## Query Data

In [ ]:
# List all people
%%cypher
MATCH (p:Person)
RETURN p.name AS name, p.age AS age, p.city AS city
ORDER BY p.age

In [ ]:
# Find who knows whom
%%cypher
MATCH (a:Person)-[:KNOWS]->(b:Person)
RETURN a.name AS person, b.name AS knows
ORDER BY a.name

In [ ]:
# Friends of friends (people Alice doesn't know directly)
%%cypher
MATCH (a:Person {name: 'Alice'})-[:KNOWS]->()-[:KNOWS]->(fof:Person)
WHERE NOT (a)-[:KNOWS]->(fof) AND a <> fof
RETURN DISTINCT fof.name AS friend_of_friend

In [ ]:
# Count relationships per person
%%cypher
MATCH (p:Person)-[r:KNOWS]->()
RETURN p.name AS person, count(r) AS knows_count
ORDER BY knows_count DESC

In [ ]:
# Shortest path between Alice and Eve
%%cypher
MATCH path = shortestPath(
    (a:Person {name: 'Alice'})-[:KNOWS*]-(b:Person {name: 'Eve'})
)
RETURN [n IN nodes(path) | n.name] AS path, length(path) AS hops

In [ ]:
# People in the same city
%%cypher
MATCH (a:Person), (b:Person)
WHERE a.city = b.city AND a.name < b.name
RETURN a.name AS person1, b.name AS person2, a.city AS city
ORDER BY city

## Access Results in Python

The last query result is stored in `_cypher` as a list of dicts.

In [ ]:
%%cypher
MATCH (p:Person)
RETURN p.name AS name, p.age AS age
ORDER BY p.age

In [ ]:
# Access the result as a list of dicts
for person in _cypher:
    print(f"{person['name']} is {person['age']} years old")

## Clean Up

In [ ]:
%%cypher
MATCH (n) DETACH DELETE n

In [ ]:
# Disconnect
%cypher --disconnect